In [ ]:
import os
import flask
import numpy as np
import keras
import tensorflow as tf
from keras.models import load_model
from flask import Flask, render_template, request
from flask_dropzone import Dropzone
from keras.preprocessing import image
app = Flask(__name__)
def init():
    global model,graph
    # load the pre-trained Keras model
    model = load_model('Malaria.h5')
    graph = tf.get_default_graph()
    
app.config.update(
    UPLOADED_PATH="/Users/lokin/OneDrive/Desktop/Major Final/CNN/upload/",
    # Flask-Dropzone config:
    DROPZONE_ALLOWED_FILE_TYPE='image',
    DROPZONE_MAX_FILE_SIZE=3,
    DROPZONE_MAX_FILES=3000,
    DROPZONE_UPLOAD_ON_CLICK=True,
    DROPZONE_REDIRECT_VIEW='completed'  # set redirect view
)
Files=[]
Results=[]


dropzone = Dropzone(app)
@app.route('/', methods=['POST', 'GET'])
def upload():
    if request.method == 'POST':
        i=0
        for key, f in request.files.items():
            if key.startswith('file'):
                f.save(os.path.join(app.config['UPLOADED_PATH'], f.filename))
                Files.append(os.path.join(app.config['UPLOADED_PATH'], f.filename))           
    return render_template('index.html')

@app.route('/completed')
def completed():
    i=0
    img=[]
    j=0
    for x in Files:
        test_image= image.load_img(x, target_size = (64, 64)) 
        test_image = image.img_to_array(test_image)
        test_image = test_image/255
        test_image = np.expand_dims(test_image, axis = 3)
        test_image = test_image.reshape(1,64, 64, 3)
#         Img = x.split('/')
#         print(Img)
#         img.append(Img[-1])
#         print(img)
        with graph.as_default():
            result = model.predict(test_image)   
        if(round(result[0][0])==0):
            Results.append("Infected")
            i=i+1
        else:
            Results.append("UnInfected")
            j=j+1
        k=(i/(i+j))*100
    print(i)
    print(j)
    print(k)
    return render_template('results.html', i=i,j=j,k=k)
    


if __name__ == '__main__':
    init()
    app.run(threaded=True)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
